In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()

In [3]:
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,4.090260e+08,0.000013,"[70814.0, 70816.0]",70811.00000,70815.50000,1.910524e+03,70774.00000,0.000565,69452.0000
1,50,ETH,False,4,3.542233e+08,0.000023,"[3581.4, 3581.5]",3581.60000,3581.45000,3.214965e+04,3579.20000,0.000615,3543.4000
2,20,ATOM,False,2,7.349990e+06,0.000013,"[10.891, 10.8924]",10.89100,10.89150,1.278815e+05,10.88700,0.000367,10.8190
3,20,MATIC,False,1,6.851621e+06,0.000013,"[0.8908, 0.89094]",0.89099,0.89085,4.975586e+06,0.89045,0.000393,0.8948
4,20,DYDX,False,1,2.722767e+06,0.000025,"[3.03864, 3.04125]",3.04000,3.03995,5.422470e+05,3.03700,0.000540,3.0591


In [4]:
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins_df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,4.090260e+08,0.000013,"[70814.0, 70816.0]",70811.00000,70815.500000,1.910524e+03,70774.00000,0.000565,69452.00000
1,50,ETH,False,4,3.542233e+08,0.000023,"[3581.4, 3581.5]",3581.60000,3581.450000,3.214965e+04,3579.20000,0.000615,3543.40000
2,20,SOL,False,2,1.476857e+08,0.000092,"[175.0726, 175.09]",175.14000,175.085000,2.778144e+05,174.91000,0.000930,172.70000
3,20,BNB,False,3,5.017977e+07,0.000103,"[606.99, 607.132]",607.03000,607.055000,1.514727e+04,606.19000,0.001320,582.46000
4,20,DOGE,False,0,3.519738e+07,0.000097,"[0.197302, 0.19735]",0.19734,0.197345,1.768640e+08,0.19708,0.001126,0.19066


In [5]:
top_60_coins = top_60_coins_df.get('name')
first_coin = top_60_coins[0]
first_coin

'BTC'

In [6]:
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import models.candles as candles
import importlib
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)

In [7]:
import time
start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
error_coins = []
for coin in top_60_coins:
    try:
        time.sleep(1)
        print(coin)
        result = info.candles_snapshot(coin, '15m', start, end)
        _candles = candles.CandleList.from_response(result)
        # _candles = candles.CandleList.save_fetch(info, coin, '15m', start, end)
        _candles.to_csv()
    except Exception as e:
        print(f"coin {coin}: not fetched")
        print(e)
        error_coins.append(coin)
        continue

BTC
ETH
SOL
BNB
DOGE
FTM
TRX
UNI
APT
ARB
MKR
kPEPE
ENA
AVAX
TIA
ORDI
OP
LINK
WIF
BCH
RUNE
RNDR
NEO
INJ
PENDLE
DOT
ATOM
NEAR
WLD
SUI
MATIC
TON
JTO
XRP
LDO
JUP
kSHIB
LTC
AAVE
GMX
MEME
ALT
FIL
SEI
FET
CFX
TAO
AR
ETC
AI
STX
ONDO
XAI
POLYX
ENS
TNSR
STRK
ADA
DYDX
W


In [18]:
# left_coins = ["DOGE", "LINK", "SEI", "BCH", "STRK", "YGG", "DOT", "UNI", "XRP", "LTC"]
# _coin = left_coins[0]
# display(start)
# display(end)
# result = info.candles_snapshot(
#     coin := _coin,
#     interval="15m",
#     startTime=start,
#     endTime=end,
# )
# result
# _candles = candles.CandleList.from_response(result)
# _df = _candles.to_df()
# _df
# _candles.to_csv()